In [13]:
import openai
import os
import pandas as pd
import time
pd.set_option('display.max_colwidth', None)

In [16]:
# Reading question and answer from the master spreadsheet

df = pd.read_csv("masters_spreadsheet_with_unique_intent_version3.csv")
df.head()

,primary_entities,secondary_entities,answer,question,unique_intents
0,admission requirements,application process,"""Please submit the following required document in order to apply:\nTranscripts from all colleges/universities attended\nDegree Conferral (Diploma or certificate)\nEnglish language assessment (Duolingo: 110, TOEFL: 85 (not accepting home edition), IELTS: 6.5, and PTE: 59.)\nResume\nStatement of Purpose\nGRE/GMAT scores (Physics+ Math only)\nLetters of recommendation (Physics+ Math only)\nAdmission page: https://www.yu.edu/katz/admissions/grad-admissions\nhttps://www.yu.edu/sites/default/files/inline-files/FINAL%20Application%20Checklist%20for%20Graduate%20STEM%20Programs%20-%20010423_0.pdf""",What are the admission requirements and how do I apply?,admissionrequirementsapplicationprocess
1,programs curriculum,research opportunities,"""Please send student program curriculum via website. (Can be found under Helpful Links under program page)\nhttps://www.yu.edu/katz/research\nhttps://www.yu.edu/katz/research-symposium-2023\nhttps://www.yu.edu/katz/research-symposium-2022""",What are the program's curriculum and research opportunities?,programscurriculumresearchopportunities
2,cost of the program,scholarships financial aid options,"""The STEM Fellows Scholarship is awarded to qualified and admitted students which reduces your tuition to $25,000 plus semesterly fees which is part of the STEM Fellows Scholarship. The additional semesterly fees that you would be responsible for can all be viewed online\nhttps://www.yu.edu/katz/stem-fellows""","What is the cost of the program, and are there any scholarships or financial aid options available?",costoftheprogramscholarshipsfinancialaidoptions
3,application deadline,admission team,"""For the Fall 2023 semester, our priority deadline is June 1st however, we encourage you to complete your application sooner, so it allows enough time for visa processing.\nAdmission page: https://www.yu.edu/katz/admissions/grad-admissions\nhttps://www.yu.edu/sites/default/files/inline-files/FINAL%20Application%20Checklist%20for%20Graduate%20STEM%20Programs%20-%20010423_0.pdf""","What is the application deadline, and when can I expect to hear back from the admission team?",applicationdeadlineadmissionteam
4,acceptance rate,program,"""Some of our STEM programs do have pre-requisites that students are required to have completed prior to the program (DAV and AI). Typically, we ask students to have a minimum of a 3.0 GPA however, we do still encourage you to apply. If we need any additional documents to review your application, we would be sure to be in touch.""","What is the acceptance rate for the program, and how competitive is it?",acceptancerateprogram


In [17]:
len(df)

143

In [18]:
def get_completion(prompt, model="gpt-3.5-turbo", retries=3):
    for _ in range(retries):
        try:
            messages = [{"role": "user", "content": prompt}]
            response = openai.ChatCompletion.create(
                model=model,
                messages=messages,
                temperature=0, # this is the degree of randomness of the model's output
            )
            return response.choices[0].message["content"]
        except openai.error.ServiceUnavailableError as e:
            print(f"Service Unavailable. Retrying after 5 seconds... Error: {e}")
            time.sleep(5)
        except openai.error.APIError as e:
            print(f"API Error. Retrying after 5 seconds... Error: {e}")
            time.sleep(5)
    raise Exception("API call failed after multiple retries.")

In [19]:
# Function for generating intents with examples for each question

def intents_examples(question):
    
    text = question
    
    prompt = f"""
Generate atleast ten similar short questions for the question given in the text

\"\"\"{text}\"\"\"
"""

    response = get_completion(prompt)
    return response

In [20]:
# Add a new column 'examples' to store the generated examples
df['examples'] = ""

# Iterate over each row in the DataFrame and generate examples for each question
for idx, row in df.iterrows():
    question = row['question']
    examples = intents_examples(question)
    df.at[idx, 'examples'] = examples

Service Unavailable. Retrying after 5 seconds... Error: The server is overloaded or not ready yet.


In [21]:
df

,primary_entities,secondary_entities,answer,question,unique_intents,examples
0,admission requirements,application process,"""Please submit the following required document in order to apply:\nTranscripts from all colleges/universities attended\nDegree Conferral (Diploma or certificate)\nEnglish language assessment (Duolingo: 110, TOEFL: 85 (not accepting home edition), IELTS: 6.5, and PTE: 59.)\nResume\nStatement of Purpose\nGRE/GMAT scores (Physics+ Math only)\nLetters of recommendation (Physics+ Math only)\nAdmission page: https://www.yu.edu/katz/admissions/grad-admissions\nhttps://www.yu.edu/sites/default/files/inline-files/FINAL%20Application%20Checklist%20for%20Graduate%20STEM%20Programs%20-%20010423_0.pdf""",What are the admission requirements and how do I apply?,admissionrequirementsapplicationprocess,"1. What are the prerequisites for admission and what is the application process?\n2. What qualifications do I need to meet in order to apply, and how can I submit my application?\n3. What are the criteria for admission, and what steps do I need to take to apply?\n4. What are the necessary documents and qualifications needed for admission, and how can I apply?\n5. What are the admission requirements, and what is the procedure for submitting an application?\n6. What qualifications and documents are needed for admission, and what is the application procedure?\n7. What are the prerequisites for admission, and what is the process for applying?\n8. What are the admission criteria, and how can I go about applying?\n9. What qualifications and documents do I need to provide for admission, and how do I apply?\n10. What are the requirements for admission, and what steps do I need to follow to apply?"
1,programs curriculum,research opportunities,"""Please send student program curriculum via website. (Can be found under Helpful Links under program page)\nhttps://www.yu.edu/katz/research\nhttps://www.yu.edu/katz/research-symposium-2023\nhttps://www.yu.edu/katz/research-symposium-2022""",What are the program's curriculum and research opportunities?,programscurriculumresearchopportunities,1. Can you provide information about the curriculum and research opportunities offered by the program?\n2. What does the program's curriculum consist of and what research opportunities are available?\n3. Could you please elaborate on the curriculum and research options provided by the program?\n4. What kind of curriculum does the program offer and what research opportunities can students pursue?\n5. Can you give me an overview of the program's curriculum and the research opportunities it provides?\n6. What are the specific courses included in the program's curriculum and what research opportunities are available to students?\n7. How does the program's curriculum align with research opportunities for students?\n8. Are there any unique or specialized research opportunities offered within the program's curriculum?\n9. Can you explain how the program's curriculum integrates research opportunities for students?\n10. What resources or support does the program provide to students interested in research within the curriculum?
2,cost of the program,scholarships financial aid options,"""The STEM Fellows Scholarship is awarded to qualified and admitted students which reduces your tuition to $25,000 plus semesterly fees which is part of the STEM Fellows Scholarship. The additional semesterly fees that you would be responsible for can all be viewed online\nhttps://www.yu.edu/katz/stem-fellows""","What is the cost of the program, and are there any scholarships or financial aid options available?",costoftheprogramscholarshipsfinancialaidoptions,1. How much does the program cost?\n2. Are there any scholarships available for the program?\n3. Are there any financial aid options for the program?\n4. What is the price of the program?\n5. Are there any grants or funding opportunities for the program?\n6. Can I apply for financial assistance for the program?\n7. Are there any

In [22]:
df.to_csv("master_spreadsheet_with_multiple_examples_version3.csv", index=False)